![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [44]:
## import different files and merge into one
import numpy as np
import pandas as pd

a_prc = pd.DataFrame(pd.read_csv('data/airbnb_price.csv'))
a_rt = pd.DataFrame(pd.read_excel('data/airbnb_room_type.xlsx', sheet_name=0))
a_lr = pd.DataFrame(pd.read_table('data/airbnb_last_review.tsv'))

merged_1 = a_prc.merge(a_rt, how='inner', on='listing_id')
airbnb = merged_1.merge(a_lr, how='inner', on='listing_id')

airbnb.head()

,listing_id,price,nbhood_full,description,room_type,host_name,last_review
0,2595,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt,Jennifer,May 21 2019
1,3831,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,July 05 2019
2,5099,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,June 22 2019
3,5178,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,June 24 2019
4,5238,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,June 09 2019


In [45]:
## how many of the listings are private rooms?


## need to do some cleaning of text first...
airbnb['room_type'] = airbnb['room_type'].str.upper()

agg = airbnb.groupby(['room_type']).count()

nb_private_rooms = 11356

## clean and change date text column to actual mm-dd-yyyy date format
airbnb[['month','day','year']] = airbnb['last_review'].str.split(' ',expand=True)
airbnb['month'] = pd.to_datetime(airbnb['month'], format='%B').dt.month
airbnb['last_review_dt'] = pd.to_datetime(airbnb[['year','month','day']])

## clean and change price column to integer
airbnb['price'] = airbnb['price'].str.split(' ').str[0]
airbnb['price'] = airbnb['price'].astype('int')


In [46]:
## what is the average listing price?
avg_price = np.round(np.mean(airbnb['price']),2)
print(avg_price)

## when are the first and last review dates?
first_reviewed = np.min(airbnb['last_review_dt'])
print(first_reviewed)

last_reviewed = np.max(airbnb['last_review_dt'])
print(last_reviewed)

## combine into a single-row Data Frame
review_dates = pd.DataFrame(
{'first_reviewed': first_reviewed, 'last_reviewed': last_reviewed, 'nb_private_rooms': nb_private_rooms, 'avg_price': avg_price}, index=[0]
)

print(review_dates)


141.78
2019-01-01 00:00:00
2019-07-09 00:00:00
  first_reviewed last_reviewed  nb_private_rooms  avg_price
0     2019-01-01    2019-07-09             11356     141.78
